In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split


# **DATA DESCRIPTION**

In [2]:
# load data and describe 
file = pd.read_csv('dirty_cafe_sales.csv')
df = pd.DataFrame(file)
df.info()
df.describe()
print(df.head())


In [3]:
# consistificate missing values to NaN
df.replace(["ERROR", "UNKNOWN", "", " "], np.nan, inplace=True)

# Identify missing values
new_missing_values = df.isnull().sum()
print("Missing values count: \n",new_missing_values)
new_missing_percent = df.isnull().sum() / len(df) * 100 
print("\nNew missing percent: \n", new_missing_percent)


In [4]:
# visualize missing data
plt.figure(figsize=(20, 12))
sns.heatmap(df.isna(), cmap='viridis', cbar=False)
plt.title('Missing data in the dataset')
plt.show()

Heatmap cho thấy cột Payment Method và Location là có đa số phần tử đều bị thiếu. Riêng cột Transaction ID thì không có giá trị bị thiếu. Nhưng cột này và Transaction Date lại không đóng góp trong quá trình huấn luyện, thậm chí gây nhiễu, nên sẽ bị bỏ sau.

In [5]:
missing_counts = df.isnull().sum()/len(df)*100
plt.figure(figsize=(10, 6))
missing_counts.plot(kind='bar', color='skyblue')
plt.title('Missing percent per Column')
plt.xlabel('Columns')
plt.ylabel('Percentage')
plt.show()

Biểu đồ cho thấy số lượng mẫu bị thiếu giá trị Payment Method và Location rất cao.

In [6]:
# count types of values for Item, Payment Method, Location
item_counts = df['Item'].value_counts().reset_index()
item_counts.columns = ['Item', 'Count']

payment_counts = df['Payment Method'].value_counts().reset_index()
payment_counts.columns = ['Payment Method', 'Count']

location_counts = df['Location'].value_counts().reset_index()
location_counts.columns = ['Location', 'Count']

# visualize the counts
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(18, 6))

# Pie chart for Item
ax1.pie(item_counts['Count'], labels=item_counts['Item'], autopct='%1.1f%%', startangle=90)
ax1.set_title('Types of Item values')

# Pie chart for Payment Method
ax2.pie(payment_counts['Count'], labels=payment_counts['Payment Method'], autopct='%1.1f%%', startangle=90)
ax2.set_title('Types of Payment Method values')

# Pie chart for Location
ax3.pie(location_counts['Count'], labels=location_counts['Location'], autopct='%1.1f%%', startangle=90)
ax3.set_title('Types of Location values')

# show
plt.tight_layout()
plt.show()



3 Biểu dồ trên thể hiện các giá trị có thể có của 3 cột Items, Payment Method và Location. Đây là 3 cột duy có dữ liệu kiểu string, cần áp dụng Label Encoding để tiện cho quá trình huấn luyện.

In [7]:
# count types of values for Item, Payment Method, Location
item_counts = df['Quantity'].value_counts().reset_index()
item_counts.columns = ['Q', 'Count']

payment_counts = df['Price Per Unit'].value_counts().reset_index()
payment_counts.columns = ['P', 'Count']

location_counts = df['Total Spent'].value_counts().reset_index()
location_counts.columns = ['T', 'Count']

# visualize the counts
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(18, 6))

# Pie chart for Item
ax1.pie(item_counts['Count'], labels=item_counts['Q'], autopct='%1.1f%%', startangle=90)
ax1.set_title('Types of Quantity values')

# Pie chart for Payment Method
ax2.pie(payment_counts['Count'], labels=payment_counts['P'], autopct='%1.1f%%', startangle=90)
ax2.set_title('Types of Price Per Unit values')

# Pie chart for Location
ax3.pie(location_counts['Count'], labels=location_counts['T'], autopct='%1.1f%%', startangle=90)
ax3.set_title('Types of Total Spent values')

# show
plt.tight_layout()
plt.show()

3 biểu đồ trên cho thấy, cột Quantiy và Price Per Unit sẽ chỉ bao gồm các giá trị cụ thể, không thực sự biến thiên nhiều. Điều này sẽ giúp cho mô hình dự đoán các giá trị này tốt hơn. Ở chiều ngược lại, cột Total Spent có sự biến thiên giá trị lớn hơn, nên kết quả sẽ không được tốt bằng 2 cột còn lại.

In [8]:
# visualize the distribution of the numerical columns
fig, ((ax1, ax2, ax3), (ax4, ax5, ax6)) = plt.subplots(2, 3, figsize=(25, 12))  # 1 row, 3 columns

df['Quantity'] = df['Quantity'].astype(float)
df['Price Per Unit'] = df['Price Per Unit'].astype(float)
df['Total Spent'] = df['Total Spent'].astype(float)
# Distribution of Quantity
sns.histplot(df['Quantity'], ax=ax1, color='skyblue')
ax1.set_title('Distribution of Quantity')

sns.histplot(df['Price Per Unit'], ax=ax2, color='coral')
ax2.set_title('Distribution of Price Per Unit')

sns.histplot(df['Total Spent'], ax=ax3, color='green')
ax3.set_title('Distribution of Total Spent')

sns.histplot(df['Item'], ax=ax4, color='gray')
ax4.set_title('Distribution of Item')

sns.histplot(df['Payment Method'], ax=ax5, color='pink')
ax5.set_title('Distribution of Payment Method')

sns.histplot(df['Location'], ax=ax6, color='purple')
ax6.set_title('Distribution of Location')


plt.show()


**Problem Price Consistency:**

Prices for menu items are consistent but may have missing or incorrect values introduced.

In [9]:
# menu prices from datacard
dicts ={
    "Coffee": 2,
    "Tea": 1.5,
    "Sandwich": 4,
    "Salad": 5,
    "Cake": 3,
    "Cookie": 1,
    "Smoothie": 4,
    "Juice": 3,
}

# search for incorrect values
incorrect_price = []
incorrect_total = []
for index, row in df.iterrows():
    item = str(row['Item'])
    if item not in dicts:
        continue
    
    if pd.isna(row['Price Per Unit']) or pd.isna(row['Total Spent']) or pd.isna(row['Quantity']):
        continue

    # price per unit check
    true_price = float(dicts.get(item))
    if float(row['Price Per Unit']) != true_price:
        incorrect_price.append(index)

    # total spent check
    quantity = float(row['Quantity'])
    total = float(row['Total Spent'])
    if total != quantity * true_price:
        incorrect_total.append(index)
print("Number of incorrect price per unit: ",len(incorrect_price))
print("Number of incorrect total spent: ", len(incorrect_total))

**--> Dữ liệu không bị sai lệch bảng giá, chỉ bị thiếu dữ liệu ở các dòng**

# **Problem Choosing: Predict quantity of item for each transaction**

# **MISSING VALUES HANDLING**

In [10]:
# Drop transaction ID, Transaction Date, Total Spent -> these columns are not useful for model training
df.drop(['Transaction ID', 'Transaction Date', 'Total Spent'], axis=1, inplace=True)
print(df.head())

In [ ]:
# split data into 2 sets: clear data and dirty data
# clear data: used to create test set (2000 samples)
# dirty data: used to create training set (8000 samples)
clear_data = df.dropna()    


print("CLEAR DATA INFO:")
print(clear_data.info())

test_data = clear_data.sample(n=2000, random_state=22520109)
train_data = df.drop(test_data.index)

print("\n\nTEST DATA INFO:")
print(test_data.info())


print("\n\nTRAIN DATA INFO:")
print(train_data.info())

## **Method 1: Drop rows with missing data**

In [12]:
train_data1 = train_data.copy()
train_data1.dropna(inplace=True)
print("\n\nTRAIN DATA1 missing values count:")
print(train_data1.info())


In [13]:
# compare the number of rows with and without missing values
rows_with_missing = train_data.isnull().any(axis=1).sum()
rows_complete = len(train_data) - rows_with_missing


labels = ['Rows with at least\n 1 Missing Values', 'Rows with No \nMissing Values']
values = [rows_with_missing, rows_complete]

plt.figure(figsize=(5, 5))
plt.bar(labels, values, color=['salmon', 'lightgreen'])
plt.title('Number of Rows with vs without \nMissing Values in Training Data')
plt.ylabel('Number of Rows')
for i, v in enumerate(values):
    plt.text(i, v + 100, str(v), ha='center')  # write the number of each bar
plt.show()

Tuy số lượng hàng bị thiếu dữ liệu ở từng cột không nhiều, nhưng khi so sánh trên toàn bộ các cột, số lượng hàng có ít nhất 1 trường dữ liệu bị trống là rất lớn (6763 hàng), nên ta nếu ta bỏ đi toàn bộ những hàng này, tập train sẽ còn lại rất ít mẫu, kết quả từ mô hình huấn luyện trên tập dữ liệu này sẽ không được tốt.

Do dữ liệu gồm các cột chứa giá trị số và các cột chứa giá trị chuỗi, nên ta sẽ dùng mô hình Linear Regression để huấn luyện dự đoán các cột có giá trị số và Logistic Regression cho các cột có giá trị chuỗi.

In [14]:
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer

# choose a column as label, others as features
def create_input_LogReg(data, selected_column):
    # print("\nCreating input for Logistic Regression model...")
    # choose label
    numeric_features = ['Quantity', 'Price Per Unit']
    string_features = ['Item', 'Payment Method', 'Location']
    if selected_column in string_features:
        string_features.remove(selected_column)
    else:
        numeric_features.remove(selected_column)

    # split data into features and label
    y = data[selected_column]
    X = data.drop(selected_column, axis=1)

    # one-hot encoding for string features and scaling for numerical features
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', StandardScaler(), numeric_features),
            ('cat', OneHotEncoder(sparse_output=False), string_features)
        ])
    X = preprocessor.fit_transform(X)
    X = pd.DataFrame(X)

    # print("Shape of data after encoding: ", X.shape)
    # print(X.columns)


    # label encoding for label
    le = LabelEncoder()
    y = le.fit_transform(y)
    y = pd.DataFrame(y)
    # print("Shape of label after encoding: ", y.shape, "\n")

    # return
    return X, y, le
    

def create_input_LinReg(data, selected_column):
    # print(f"\nCreating input for {selected_column}...")

    # choose label
    numeric_features = ['Quantity', 'Price Per Unit']
    string_features = ['Item', 'Payment Method', 'Location']
    if selected_column in string_features:
        string_features.remove(selected_column)
    else:
        numeric_features.remove(selected_column)

    # split data into features and label
    y = data[selected_column]
    X = data.drop(selected_column, axis=1)
    # X.drop(['Payment Method', 'Location', 'Item'], axis=1, inplace=True)
    # print(X.columns)

    # one-hot encoding for string features and scaling for numerical features
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', StandardScaler(), numeric_features),
            ('cat', OneHotEncoder(sparse_output=False), string_features)
        ])
    X = preprocessor.fit_transform(X)
    X = pd.DataFrame(X)

    # return mean of y 
    mean_y = y.astype(float).mean()


    # print("Shape of data after encoding: ", X.shape)

    # return
    return X, y, mean_y


# X_train, y_train = create_input_LogReg(train_data1, select)
# X_test, y_test = create_input_LogReg(test_data, select)


select = "Quantity"
X_train, y_train, _ = create_input_LinReg(train_data1, select)
X_test, y_test, _ = create_input_LinReg(test_data, select)

print(X_train.head())
print(X_train.shape)

print(y_train.head())

# embeded columns order: Price Per Unit, Item, Payment Method, Location



### **Train and evaluate models**

In [ ]:
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.metrics import accuracy_score, root_mean_squared_error

# Logistic Regression model
def logistic_regression(selected_column, train_data, max_iter=1000):
    X_train, y_train, le_train = create_input_LogReg(train_data, selected_column)
    X_test, y_test, le_test = create_input_LogReg(test_data, selected_column)
    model = LogisticRegression(max_iter=max_iter)
    model.fit(X_train, y_train.values.ravel())
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Accuracy for {selected_column} ", accuracy)

def linear_regression(selected_column, train_data):
    X_train, y_train, y_train_mean = create_input_LinReg(train_data, selected_column)
    X_test, y_test, y_test_mean = create_input_LinReg(test_data, selected_column)

    print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
    print(f"X_test shape: {X_test.shape}, y_test shape: {y_test.shape}")

    model = LinearRegression()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    mse = root_mean_squared_error(y_test, y_pred)
    print(f"RMSE for {selected_column} ", mse)
    print(f"\tMean of {selected_column} in train data: ", y_train_mean)
    print(f"\tMean of {selected_column} in test data: ", y_test_mean)
    print(f"\tRMSE/y_test_mean: {(mse/y_test_mean):.2f}, RMSE/y_train_mean: {(mse/y_train_mean):.2f}")

# string_columns = ['Item', 'Payment Method', 'Location']
# numeric_columns = ['Price Per Unit', 'Quantity', 'Total Spent']
# for column in string_columns:
#     logistic_regression(column, train_data1, 2000)

# print("\n")
# for column in numeric_columns:
#     linear_regression(column, train_data1

linear_regression('Quantity', train_data1)




### **Kết luận:**

<!-- - Cột Item: giá trị này tạm chấp nhận được. Do mỗi nhãn trong Item đều tương ứng với 1 giá trị giá (có trùng nhau), có thể tính toán được khi có giá trị Quantity và Total Spent, nên mô hình có thể học được mối quan hệ này, mặc dù dữ liệu huấn luyện quá ít so với dữ liệu test -->
<!-- - Cột Payment Method và Location: 2 cột này không hề có quan hệ rõ ràng với các trường dữ liệu nên kết quả thu được thấp hơn chủ yếu là do sự chênh lệch số lượng giữa tập train và tập test -->
<!-- - Cột Price Per Unit: Tỉ lệ RMSE/mean đạt được gần như là 0, cho thấy mô hình gần như hoàn hảo. Điều này có thể lý giải do mối quan hệ *Total Spent = Price Per Unit * Quantity* được thể hiện chặt chẽ trong các samples. Thêm vào đó, các giá trị Price Per Unit trong dữ liệu có rất ít biến thiên cũng góp phần vào sự hoàn hảo của mô hình -->
- RMSE cho cột Quantity: Tỉ lệ RMSE/mean = 0.48 thể hiện sai số trung bình của mô hình chiếm 48% so với dữ liệu thực tế, đây là một kết quả rất tệ. Có thể thấy, sau khi bỏ đi cột Total Spent, mối tương quan *Total Spent = Quantity * Price Per Unit* đã không còn, cộng thêm việc dữ liệu của cột Quantity không có phân phối chuẩn, nên model linear regression không hoạt động tốt. Việc bỏ đi các dòng bị thiếu dữ liệu khiến kích thước tập train nhỏ hơn test, điều này cũng làm cho độ lỗi tăng lên do mô hình chưa có nhiều dữ liệu để học.

## **Method 2: Mean/Median Imputation**

Do có một số cột dữ liệu là kiểu chuỗi nên ta không thể dùng Mean hay Median, thay vào đó, ta dùng Mode

In [16]:
# make a new copy of train data and fill missing values
train_data2 = train_data.copy()
# print(train_data2.info())


string_columns = ['Item', 'Payment Method', 'Location']
numeric_columns = ['Price Per Unit', 'Quantity']
print("Filling string columns with mode and numeric columns with mean...")
for column in string_columns:
    print(f"\t{column}.mode() = {train_data2[column].mode()[0]}")
    train_data2[column] = train_data2[column].fillna(train_data2[column].mode()[0])

for column in numeric_columns:
    train_data2[column] = pd.to_numeric(train_data2[column], errors='coerce')
    # replace_value = train_data2[column].median()
    replace_value = train_data2[column].mean()
    print(f"\t{column}.mean() = {replace_value}")
    train_data2[column] = train_data2[column].fillna(replace_value)

missing_values_count = train_data2.isna().sum()
print("\n\nTRAIN DATA2 missing values count:")  
print(missing_values_count)

### **Train and evaluate models**

In [17]:
# for column in string_columns:
#     logistic_regression(column, train_data2, 2000)

# for column in numeric_columns:
#     linear_regression(column, train_data2)

linear_regression('Quantity', train_data2)

### **Kết luận:**

<!-- Nhìn chung, kết quả của phương pháp này dành cho các cột có giá trị chuỗi thấp hơn đáng kể so với phương pháp Drop rows with missing data, trong khi các cột có giá trị số thì vẫn ở mức khá tốt mặc dù có thấp hơn một chút. 

Lý giải cho điều này, đối với các cột chứa dữ liệu chuỗi, đặc biệt là cột Item (bị giảm đáng kể accuracy), khi điền các vị trí bị thiếu bằng Mode của cột, các giá trị này hầu như sẽ không đúng, những giá trị sai lệch này sẽ ảnh hưởng đến bảng giá trong menu khiến cho giá của sản phẩm bị lẫn lộn, model không thể học được sự liên quan giữa Item và Price Per Unit, khiến cho Acccuracy bị giảm mạnh. Còn lại các cột Payment Method và Location, do không có mối quan hệ rõ ràng nào với các trường dữ liệu khác, cộng thêm phần lớn các dòng bị thiếu dữ liệu, nên việc điền bằng Mode cũng làm nhiễu đi thông tin, khiến kết quả còn thấp hơn cả phương pháp trước đó mặc dù có nhiều dữ liệu huấn luyện hơn.

Đối với các cột chứa dữ liệu số, kết quả vẫn ở mức tốt do có quan hệ giữa các cột Quantity, Payment Method và Price Per Unit. Nhưng kết quả có phần giảm nhẹ so với phương pháp trước đó, điều này là do tập train nhiều dữ liệu hơn, nhưng lại là các giá trị được nội suy, không đảm bảo tính đúng đắn, làm nhiều quá trình học được mối quan hệ giữa các cột. -->

Ở phương pháp này số lượng mẫu của tập train đã có nhiều hơn, nhưng thực tế thì phần lớn chúng là các giá trị được điền vào bằng giá trị mode/mean. Những giá trị này đa phần sẽ không đúng và gây nhiễu cho việc học của mô hình. Vậy nên kết quả thu được vẫn chưa được tốt, mặc dù có cải thiện 1%.

Điều tương tự cũng xảy ra khi thay Mean bằng Median.

## **Method 3: Advanced Techniques**

Dùng các mô hình máy học để dự đoán các giá trị khuyết.


***Sự khác biệt cơ bản giữa Custom KNN và KNNImputer:***

- Custom KNN: Chia tập train thành 2 phần: complete (không có NaN) và missing (chỉ chứa các hàng có ít nhất 1 NaN). Dùng tập complete để train KNN, sau đó dùng model KNN đã được huấn luyện để dự đoán các giá trị bị thiếu trong tập missing. Vấn đề gặp phải là các hàng trong tập missing có khả năng bị *thiếu dữ liệu ở các features columns* (xảy ra khi hàng đó thiếu trên 2 trường dữ liệu). Cách xử lý: Điền tạm các giá trị bị thiếu trong các features column bằng Mean/Mode.

- KNNImputer: dùng thằng tập train ban đầu, không cần điền tạm. Khi dùng lệnh fit, KNNImputer sẽ lặp qua từng dòng trong tập dữ liệu được, với mỗi giá trị bị thiếu ở 1 cột, nó sẽ tìm *k* hàng ở gần nhất trong không gian để tham chiếu, sau đó lấy trung bình giá trị ở *k* hàng đó và điền vào trường bị thiếu trong hàng mục tiêu. Không gặp phải vấn đề thiếu dữ liệu ở các features columns vì KNNImputer chỉ thực hiện tính toán đối với các hàng khác có trùng các cột không bị thiếu dữ liệu với hàng mục tiêu. Ví dụ hàng mục tiêu thiếu các giá trị x1, x2 và có đủ các giá trị từ x3 đến x8, và ta đang cần điền vào x1, thì KNNImputer sẽ tìm kiếm các hàng có giá trị x1, thiếu giá trị x2 và có đủ các giá trị từ x3 đến x8 để tính toán khoảng cách trong không gian Euclidean với hàng mục tiêu, sau đó chọn ra *k* hàng gần nhất, lấy trung bình giá trị x1 và điền vào x1 đang bị thiếu của hàng mục tiêu.

### **Custom KNN**

- Ý tưởng: Do dữ liệu có giá trị bị khuyết nằm ở nhiều cột, và các dòng có thể có 1 hoặc nhiều cột bị thiếu, nên ta sẽ tách từ train_data ra 1 tập các hàng không bị thiếu giá trị dùng để huấn luyện mô hình KNN, sau đó dùng chính mô hình KNN đã huấn luyện này để dự đoán các giá trị bị thiếu (từng cột một) trong phần còn lại của train_data.


In [18]:
# demonstrate how many rows have missing values
missing_values_count = train_data.isna().sum(axis=1)
missing_type = [0,0,0,0,0,0,0,0]

for index, row in train_data.iterrows():
    row_missing_count = row.isna().sum()
    missing_type[row_missing_count] += 1



titles = ["no missing", "1 missing", "2 missing", "3 missing", "4 missing", "5 missing", "6 missing", "7 missing"]

plt.figure(figsize=(18, 8))
plt.bar(titles, missing_type)
for i, v in enumerate(missing_type):
    plt.text(i, v + 50, str(v), ha='center')  # write the number of each bar
plt.title("Number of rows with missing values")
plt.xlabel("Number of missing values")    
plt.ylabel("Number of rows")
plt.show()

Như ta có thể thấy trên biểu đồ, có một số lượng không nhỏ những hàng bị thiếu trên 2 trường dữ liệu.

In [19]:
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier
# Convert numeric columns to float
train_data['Price Per Unit'] = pd.to_numeric(train_data['Price Per Unit'], errors='coerce')
train_data['Quantity'] = pd.to_numeric(train_data['Quantity'], errors='coerce')
# train_data['Total Spent'] = pd.to_numeric(train_data['Total Spent'], errors='coerce')
# Tách train_data thành 2 phần
train_data_complete = train_data.dropna()
train_data_missing = train_data[train_data.isna().any(axis=1)]

print("Train data complete: ", train_data_complete.shape)
print("Train data missing: ", train_data_missing.shape) 


In [20]:
# impute for numeric columns
def knn_reg(train_data_complete, train_data_missing, selected_column):
    numeric_columns = ['Price Per Unit', 'Quantity']
    string_columns = ['Item', 'Payment Method', 'Location']
    # create a copy of train_data_missing, doesn't change the original data
    df_missing = train_data_missing.copy()
    print(f"Current selected column: {selected_column}")

    # fill features cols in train_data_missing with temp value using mean
    for column in numeric_columns:
        if column == selected_column:
            continue
        df_missing[column] = pd.to_numeric(df_missing[column], errors='coerce')
        df_missing[column] = df_missing[column].fillna(df_missing[column].mean())

    # fill features cols in train_data_missing with temp value using mode
    for column in string_columns:
        if column == selected_column:
            continue
        # df_missing[column].fillna({column:train_data_complete[column].mode()[0]}, inplace=True)
        df_missing[column] = df_missing[column].fillna(df_missing[column].mode()[0])

    # create model input for KNN
    X_train, y_train, y_train_mean = create_input_LinReg(train_data_complete, selected_column)
    knn = KNeighborsRegressor(n_neighbors=5)
    knn.fit(X_train, y_train)

    # Predict missing values in target_col of train_data_missing
    mask = df_missing[selected_column].isna()
    if mask.any():
        X_missing, _, _ = create_input_LinReg(df_missing[mask], selected_column) # create features for missing values
        predicted = knn.predict(X_missing)
        df_missing.loc[mask, selected_column] = predicted

    return df_missing


# impute for string columns
def knn_clf(train_data_complete, train_data_missing, selected_column):
    string_columns = ['Item', 'Payment Method', 'Location']
    numeric_columns = ['Price Per Unit', 'Quantity']
    df_missing = train_data_missing.copy()
    print(f"Current selected column: {selected_column}")
    try:
        # fill features cols in train_data_missing with temp value using mode
        for column in string_columns:
            if column == selected_column:
                continue
            # df_missing[column].fillna({column:train_data_complete[column].mode()[0]}, inplace=True)
            df_missing[column] = df_missing[column].fillna(df_missing[column].mode()[0])

        # fill features cols in train_data_missing with temp value using mean
        for column in numeric_columns:
            if column == selected_column:
                continue
            df_missing[column] = pd.to_numeric(df_missing[column], errors='coerce')
            # df_missing[column].fillna({column:train_data_complete[column].mean()}, inplace=True)
            df_missing[column] = df_missing[column].fillna(df_missing[column].mean())
    except Exception as e:
        print(e)
        print("\tCurrent selected column: ", selected_column)
        print(df_missing)
    
    # create model input for KNN
    X_train, y_train, le = create_input_LogReg(train_data_complete, selected_column)
    knn = KNeighborsClassifier(n_neighbors=5)
    knn.fit(X_train, y_train.values.ravel())

    # Predict missing values in target_col of train_data_missing
    mask = df_missing[selected_column].isna()
    if mask.any():
        X_missing, _, _ = create_input_LogReg(df_missing[mask], selected_column) # create features for missing values
        predicted = knn.predict(X_missing)
        df_missing.loc[mask, selected_column] = le.inverse_transform(predicted) # inverse values to string labels

    return df_missing


# IMPUTING
string_columns = ['Item', 'Payment Method', 'Location']
numeric_columns = ['Price Per Unit', 'Quantity']
filled_string_train = train_data_missing.copy()
filled_numeric_train = train_data_missing.copy()
for column in string_columns:
    # drop selected column
    filled_string_train.drop(column, axis=1, inplace=True)
    temp_string_train = knn_clf(train_data_complete, train_data_missing, column)
    filled_string_train[column] = temp_string_train[column] # add selected column back

for column in numeric_columns:
    # drop selected column
    filled_numeric_train.drop(column, axis=1, inplace=True)
    temp_numeric_train = knn_reg(train_data_complete, train_data_missing, column)
    filled_numeric_train[column] = temp_numeric_train[column] # add selected column back


# merge filled data
filled_train = filled_string_train.drop(numeric_columns, axis=1)
filled_train = pd.concat([filled_train, filled_numeric_train[numeric_columns]], axis=1)
custom_knn_train_data = pd.concat([train_data_complete, filled_train])

# check missing values
missing_values_count = filled_numeric_train.isna().sum()
print("\nFILLED NUMERIC TRAIN missing values count:")
print(missing_values_count)

missing_values_count = filled_string_train.isna().sum()
print("\nFILLED STRING TRAIN missing values count:")
print(missing_values_count)

missing_values_count = custom_knn_train_data.isna().sum()
print("\nTRAIN DATA FILLED missing values count:")
print(missing_values_count)

In [21]:
print(custom_knn_train_data.head())

#### **Train and evaluate:**

In [22]:
# for column in string_columns:
#     logistic_regression(column, custom_knn_train_data, 2000)

# for column in numeric_columns:
#     linear_regression(column, custom_knn_train_data)
linear_regression('Quantity', custom_knn_train_data)

#### **Kết luận:**

- Kết quả cho thấy tỉ lệ lỗi vẫn không đổi so với phương pháp 1, thậm chí còn thấp hơn phương pháp 2. Mặc dù mô hình KNN được huấn luyện trên một tập dữ liệu đầy đủ nhằm dự đoán các giá trị bị mất trên phần còn lại của tập train, nhưng khi dự đoán, tồn tại những mẫu bị thiếu nhiều giá trị features, và các giá trị thiếu này lại được điền bằng mode/mean nên kết quả không cải thiện nhiều so với các phương pháp trước.

### **KNNImputer**

Sử dụng thư viện KNNImputer để điền vào các chỗ thiếu.

In [23]:
# import and use KNNImputer
from sklearn.impute import KNNImputer
import pandas as pd

# init KNNImputer
imputer = KNNImputer(n_neighbors=2, weights="uniform")

# prepare data for KNNImputer
string_columns = ['Item', 'Payment Method', 'Location']
imputer_train_data = train_data.copy()
print("Data for KNNImputer:", imputer_train_data.shape)

# encoding string columns with one-hot encoding
label_encoders = {}
len_classes = {} 
for column in string_columns:
    le = LabelEncoder()
    imputer_train_data[column] = le.fit_transform(imputer_train_data[column])
    label_encoders[column] = le
    len_classes[column] = len(le.classes_)
    print(f"Label encoder for {column}: {le.classes_}")
    print(f"\tlen = {len(le.classes_)}")

# convert encoded NaN values to np.nan cause LabelEncoder encoded NaN values
for column in string_columns:
    value_to_replace = len_classes[column]-1
    imputer_train_data[column] = imputer_train_data[column].replace(value_to_replace, np.nan)

# check is converted
print("\nCheck if converted to NaN:")
print(imputer_train_data.head())



# impute missing values
KNNimputer_train_data = pd.DataFrame(imputer.fit_transform(imputer_train_data), columns=imputer_train_data.columns)


# inverse label encoding for string columns
for column in string_columns:
    le = label_encoders[column]
    KNNimputer_train_data[column] = le.inverse_transform(KNNimputer_train_data[column].astype(int))

print("\nDữ liệu sau khi điền bằng KNN:")
missings = KNNimputer_train_data.isna().sum()
print(missings)
print(KNNimputer_train_data.head())

#### **Train and evaluate**

In [24]:
# for column in string_columns:
#     logistic_regression(column, KNNimputer_train_data, 2000)

# for column in numeric_columns:
#     linear_regression(column, KNNimputer_train_data)

linear_regression('Quantity', KNNimputer_train_data)

#### **Kết luận:**

- Kết quả của mô hình vẫn không thay đổi mạnh. Cách hoạt động của KNN sẽ bỏ qua những hàng có số lượng features bị thiếu khác với cột đích, nên số lượng neighbours hợp lệ mà thuật toán tìm thấy trên tập dữ liệu này không còn nhiều, vì thế cũng không cải thiện được kết quả.

### **MICE**

MICE coi mỗi cột có giá trị thiếu như một biến phụ thuộc và dùng các cột khác làm đặc trưng để dự đoán giá trị thiếu thông qua mô hình hồi quy (thường là hồi quy tuyến tính).

Quá trình được thực hiện tuần tự (chained) qua từng cột và lặp lại nhiều lần để cải thiện kết quả.

In [25]:
from fancyimpute import IterativeImputer
import pandas as pd

# prepare data for MICE
MICE_train_data = train_data.copy()
string_columns = ['Item', 'Payment Method', 'Location']
numeric_columns = ['Price Per Unit', 'Quantity']
print("Data for MICE:", MICE_train_data.shape)


# encoding string columns with one-hot encoding
label_encoders = {}
len_classes = {} 
for column in string_columns:
    le = LabelEncoder()
    MICE_train_data[column] = le.fit_transform(MICE_train_data[column])
    label_encoders[column] = le
    len_classes[column] = len(le.classes_)
    print(f"Label encoder for {column}: {le.classes_}")
    print(f"\tlen = {len(le.classes_)}")

# convert encoded NaN values to np.nan cause LabelEncoder encoded NaN values
for column in string_columns:
    value_to_replace = len_classes[column]-1
    MICE_train_data[column] = MICE_train_data[column].replace(value_to_replace, np.nan)

# check is converted
print("\nCheck if converted to NaN:")
print(MICE_train_data.head())


# init MICE
imputer = IterativeImputer(max_iter=50, random_state=22520109)

# impute
MICE_imputed_train_data = pd.DataFrame(imputer.fit_transform(MICE_train_data), columns=MICE_train_data.columns)

# inverse label encoding for string columns
for column in string_columns:
    le = label_encoders[column]
    MICE_imputed_train_data[column] = le.inverse_transform(MICE_imputed_train_data[column].astype(int))

print("\nDữ liệu sau khi điền bằng MICE:")
missings = MICE_imputed_train_data.isna().sum()
print(missings)
print(MICE_imputed_train_data.head())

#### **Train and evaluate**


In [26]:
# for column in string_columns:
#     logistic_regression(column, MICE_imputed_train_data, 2000)

# for column in numeric_columns:
#     linear_regression(column, MICE_imputed_train_data)
linear_regression('Quantity', MICE_imputed_train_data)

#### **Kết luận**

- Độ lỗi của mô hình không giảm, thậm chí còn tăng thêm 1% so với KNNImputer. Điều này thể hiện mức độ ảnh hưởng của số lượng data bị mất. Dù có thực hiện các phương pháp giải quyết dữ liệu mất phức tạp hay đơn giản thì kết quả đem lại vẫn không có sự khác biệt.

# **TỔNG KẾT**
Trong số các phương pháp xử lý dữ liệu bị mất đã sử dụng, phương pháp *Custom KNN* và *MICE* cho ra hiệu quả tốt nhất, nhưng cũng không cách biệt quá lớn đối với các phương pháp khác. Nguyên nhân chủ yếu là do bộ dataset này bị thiếu thông tin không chỉ ở một cột, mà dữ liệu bị mất nằm rải rác ở tất cả các cột (trừ cột Transaction ID, nhưng không đem lại thông tin cho mô hình máy học). Các cột Location và Payment Method lại mất quá nhiều, khiến thông tin học được không đáng kể, hiểu quả mô hình thấp. Sau khi bỏ đi quan hệ ràng buộc giữa các cột dữ liệu số,  khả năng học của mô hình giảm đáng kể

Phương pháp *Dropping rows with missing values* gây ra sự mất mát thông tin đặc biệt lớn đối với bộ dữ liệu này (mất khoảng 6000/10000 mẫu, hơn một nửa số mẫu). Nhưng kết quả chung của phương pháp này lại không quá cách biệt, bởi vì dữ liệu bị thiếu quá lớn, tập trung ở các cột không có mối quan hệ rõ ràng, nên dù được tận dụng lại bằng các phương pháp cao cấp thì vẫn không thể đem lại giá trị, thậm chí còn có thể còn gây nhiễu. 

Việc mất mát dữ liệu đã gây ảnh hưởng rất lớn đến chất lượng thông tin mà dataset mang lại.